# Lecture 3 (1/2): Constraint Programming
This notebook demonstrates how constraint programming works. We use the `python-constraint` library.

# Team making
Let's say we have 20 students. We want to make groups of 1 or 2 students for their assignment.
Formulate this as a constraint programming problem.

In [1]:
from constraint import *
from itertools import product

In [2]:
num_students = 20
students = list(range(num_students))
partners = list(range(num_students))
print(f"Students: {students}")
print(f"Partner: {partners}")

Students: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Partner: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [3]:
sp = list(product(students, partners))
print(f"Pairs: {sp[:10]}...")

Pairs: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)]...


Using a constraint solver requires declaration of a problem. We then specify variables that are handled in that probelm.

In [11]:
problem = Problem()
problem.addVariables(sp, [0, 1])

In [12]:
# A student can pick only one partner
for s in students:
    pairs = [(s, p) for p in partners]
    problem.addConstraint(ExactSumConstraint(1), pairs)

Let's see how to obtain a solution so far. You can either get one solution using `problem.getSolution()` or all the solutions using `problem.getSolutions()` (or `problem.getSolutionIter()` which returns an iterator).

In [13]:
# Get one solution
sol = problem.getSolution()
print(list(filter(lambda item: item[1] == 1, sol.items())))

[((0, 0), 1), ((1, 0), 1), ((2, 0), 1), ((3, 0), 1), ((4, 0), 1), ((5, 0), 1), ((6, 0), 1), ((7, 0), 1), ((8, 0), 1), ((9, 0), 1), ((10, 0), 1), ((11, 0), 1), ((12, 0), 1), ((13, 0), 1), ((14, 0), 1), ((15, 0), 1), ((16, 0), 1), ((17, 0), 1), ((18, 0), 1), ((19, 0), 1)]


Take a look at the first tuple, `((0,0), 1)`. The first element `(0, 0)` is a tuple of student ids. The second element `1` indicates the value of the variable. Together, it indicates that $X_{0, 0} = 1$. That is, the student 0 is partnering up with himself/herself.

See the second tuple, `(1, 0), 1`. This indicates that the student 1 is partnering up with student 0. But as we have seen, student 0 is already pairing up with themselves. So such solution should be prohibited. Let's add another constraint to prevent this value to be a solution.

In [14]:
# Partners should be symmetric
for s in students:
    for p in partners:
        if s < p:
            pairs = [(s, p), (p, s)]
            problem.addConstraint(AllEqualConstraint(), pairs)

Here, I am adding a constraint to force that $X_{i,j}$ is equal to the value of $X_{j, i}$.

Let's say some students have already formed pairs. We can create a list of such pairs $(i, j)$ and add constraints to enforce $X_{i, j}=1$.

In [15]:
# Some students have formed a pair already
fixed_pairs = [(0, 1), (2, 3), (5, 6), (7, 9), (10, 12), (14, 17), (18, 19)]

problem.addConstraint(ExactSumConstraint(len(fixed_pairs)), fixed_pairs)

In [16]:
# Get all feasible solutions
solutions = problem.getSolutions()
print(f"The size of feasible solutions is: {len(solutions)}")

for sol in solutions:
    sol = list(filter(lambda item: item[1] == 1, sol.items()))
    print(sol)

The size of feasible solutions is: 76
[((0, 1), 1), ((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((1, 0), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 16), 1), ((16, 15), 1)]
[((0, 1), 1), ((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((1, 0), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 15), 1), ((16, 16), 1)]
[((0, 1), 1), ((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((1, 0), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((4, 8), 1), ((8, 4), 1), ((11, 15), 1), ((15, 11), 1), ((13, 16), 1), ((16, 13), 1)]
[((0, 1), 1), ((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((1, 0), 1), ((3, 2),

In [20]:
for sol in solutions:
    sol = list(filter(lambda item: item[1] == 1, sol.items()))
    filtered = list(filter(lambda item: (item[0] not in fixed_pairs) and ((item[0][1], item[0][0]) not in fixed_pairs), sol))
    print(filtered)


[((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 16), 1), ((16, 15), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 15), 1), ((16, 16), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 15), 1), ((15, 11), 1), ((13, 16), 1), ((16, 13), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 15), 1), ((15, 11), 1), ((13, 13), 1), ((16, 16), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 16), 1), ((16, 11), 1), ((13, 15), 1), ((15, 13), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 16), 1), ((16, 11), 1), ((13, 13), 1), ((15, 15), 1)]
[((4, 8), 1), ((8, 4), 1), ((13, 15), 1), ((15, 13), 1), ((11, 11), 1), ((16, 16), 1)]
[((4, 8), 1), ((8, 4), 1), ((13, 16), 1), ((16, 13), 1), ((11, 11), 1), ((15, 15), 1)]
[((4, 8), 1), ((8, 4), 1), ((15, 16), 1), ((16, 15), 1), ((11, 11), 1), ((13, 13), 1)]
[((4, 8), 1), ((8, 4), 1), ((11, 11), 1), ((13, 13), 1), ((15, 15), 1), ((16, 16), 1)]
[((4, 11), 1), ((11, 4), 1), ((8, 13), 1), ((13, 8), 1), ((15, 16), 1), ((16, 15), 1)]
[((4, 11), 1), ((11, 4), 1), ((8, 13), 1), 

# Design Exploration

In [23]:
from constraint import *

problem = Problem()

font_sizes = [12, 16, 20, 24, 28, 32]
color_range = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


problem.addVariable('h1-red', color_range)
problem.addVariable('h1-size', font_sizes)


problem.addConstraint(lambda h1_r: 20 <= h1_r <= 80, ['h1-red'])
problem.addConstraint(lambda h1_size: 16 <= h1_size <= 28, ['h1-size'])

In [25]:
print(f"Number of solutions: {len(problem.getSolutions())}")
sol_iter = problem.getSolutionIter()
for i, sol in enumerate(sol_iter):
    print(sol)
    # if i >= 5:
    #     break

Number of solutions: 28
{'h1-size': 28, 'h1-red': 80}
{'h1-size': 28, 'h1-red': 70}
{'h1-size': 28, 'h1-red': 60}
{'h1-size': 28, 'h1-red': 50}
{'h1-size': 28, 'h1-red': 40}
{'h1-size': 28, 'h1-red': 30}
{'h1-size': 28, 'h1-red': 20}
{'h1-size': 24, 'h1-red': 80}
{'h1-size': 24, 'h1-red': 70}
{'h1-size': 24, 'h1-red': 60}
{'h1-size': 24, 'h1-red': 50}
{'h1-size': 24, 'h1-red': 40}
{'h1-size': 24, 'h1-red': 30}
{'h1-size': 24, 'h1-red': 20}
{'h1-size': 20, 'h1-red': 80}
{'h1-size': 20, 'h1-red': 70}
{'h1-size': 20, 'h1-red': 60}
{'h1-size': 20, 'h1-red': 50}
{'h1-size': 20, 'h1-red': 40}
{'h1-size': 20, 'h1-red': 30}
{'h1-size': 20, 'h1-red': 20}
{'h1-size': 16, 'h1-red': 80}
{'h1-size': 16, 'h1-red': 70}
{'h1-size': 16, 'h1-red': 60}
{'h1-size': 16, 'h1-red': 50}
{'h1-size': 16, 'h1-red': 40}
{'h1-size': 16, 'h1-red': 30}
{'h1-size': 16, 'h1-red': 20}


In [26]:
problem.reset()